In [1]:
import numpy as np
import pandas as pd
import preprocess, topic_model

[nltk_data] Error loading stopwords: <urlopen error unknown url type:
[nltk_data]     https>
[nltk_data] Error loading punkt: <urlopen error unknown url type:
[nltk_data]     https>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     unknown url type: https>
[nltk_data] Error loading stopwords: <urlopen error unknown url type:
[nltk_data]     https>


In [2]:
path = 'test_cases\singlelinetestcase3.json'

In [3]:
data = pd.DataFrame(pd.read_json(path))
data.reset_index(inplace = True)
    

In [4]:
utterances = data['utt'][0]
speakers = data['speaker'][0]

In [5]:
questions, answers, q_indices, a_indices, qa_pairs = preprocess.question_count(utterances, speakers)


Current speaker: JOHN DONVAN, host

Current speaker: JOHN DONVAN, host

Current speaker: JOHN DONVAN, host

Current speaker: JOHN DONVAN, host

Current speaker: JOHN DONVAN, host

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR CHAZ ALLEN
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR TOMMY BATTLE
Status: response

Current speaker: JOHN DONVAN, host

Current speaker: MAYOR TOMMY BATTLE
Status: response

Current speaker: MAYOR TOMMY BATTLE
Status: response

Curre

In [6]:
len(questions)

65

In [7]:
len(answers)

69

In [8]:
len(qa_pairs.items())

50

In [9]:
question_column = qa_pairs.keys()
answer_column = qa_pairs.values()

qa_df = pd.DataFrame(columns = ['questions', 'answers'])
qa_df['questions'] = question_column
qa_df['answers'] = answer_column

In [10]:
qa_df.head()

,questions,answers
0,"So joining us now from Newton, Iowa by phone, ...","Well, when they finally shut down, we had a ch..."
1,You you had people who3 were interested in may...,"Oh, you bet. You bet."
2,A couple of things. Did Maytag give you warnin...,"Not really Maytag, but when they were sold to ..."
3,"So some of it went overseas, then?",You bet.
4,"What about, though - I'm wondering if there wa...","No. I think a lot of people - it was. I mean, ..."


# Preprocessing the questions for topic modelling

In [11]:
from preprocess import create_pipeline
import spacy
from nltk.corpus import stopwords




In [12]:
lemmatizer_model = spacy.load('en_core_web_sm', disable = ['ner', 'parser'])
stopword_eng = stopwords.words('english')
contractions_dict = {"aren t" : "are not", "can t" : "cannot", "couldnt":"could not", "couldn t" : "could not", "didn t" : "did not", "doesn t" : "does not", "don t" : "do not", "haven t" : "have not", "hasn t" : "has not" , "hadn t" : "had not", "i m" : "i am", "i ve" : "i have", "isn t" : "is not", "it's" : "it is" , "mustn t" : "must not", "shouldn t" : "should not", "wasn t" : "was not", "weren t" : "were not", "wouldn t" : "would not", "won t":"will not", "you re" : "you are", "you ll" : "you will" , "we ll" : "we will", "you ve" : "you have", "we ve" : "we have"}

In [13]:
qa_df['regular_split'] = qa_df['questions'].apply(lambda x: create_pipeline(x, ['norm', 'tags'], relevant_tags = ['NN', 'VB', 'JJ', 'NNP', 'NNS', 'JJR'], general_category = False))
qa_df['Preprocessed_Question'] = qa_df['questions'].apply(lambda x: create_pipeline(x, ['norm', 'ctrc', 'tags', 'stop', 'lemm', 'norm'], relevant_tags = ['NN', 'VB', 'JJ', 'RB'], stopwords=stopword_eng, lemmatizer = lemmatizer_model, contractions = contractions_dict, general_category = True, keep_nonascii = False, lowercase = True, word_length = 3))

In [14]:
qa_df.head()

,questions,answers,regular_split,Preprocessed_Question
0,"So joining us now from Newton, Iowa by phone, ...","Well, when they finally shut down, we had a ch...","[newton, iowa, phone, mayor, chaz, allen, hunt...","[join, newton, iowa, phone, mayor, chaz, allen..."
1,You you had people who3 were interested in may...,"Oh, you bet. You bet.","[people, interested, fashion, other]","[people, interested, maybe, come, help, fashion]"
2,A couple of things. Did Maytag give you warnin...,"Not really Maytag, but when they were sold to ...","[couple, things, give]","[couple, thing, maytag, give, warning, come]"
3,"So some of it went overseas, then?",You bet.,[],"[go, overseas]"
4,"What about, though - I'm wondering if there wa...","No. I think a lot of people - it was. I mean, ...","[i, sense, betrayal, year, history, i, people,...","[wonder, sense, betrayal, give, year, history,..."


In [15]:
count_words = np.sum(qa_df['questions'].apply(lambda x: preprocess.word_count(x)))
print("Total word count:", count_words)

Total word count: 1366


In [16]:
trigrams, _ = preprocess.generate_ngrams(qa_df, 'regular_split', n = 3, min_pmi = 10, frequency = 1, best_number = 25, stopwords_list = stopword_eng)
bigrams, _ = preprocess.generate_ngrams(qa_df, 'regular_split', n = 2, min_pmi = 8, frequency = 1, best_number = 5, stopwords_list = stopword_eng)


                               ngram        pmi
0          (affairs, author, caught)  17.469419
16              (idea, first, place)  17.469419
1           (author, caught, middle)  17.469419
30  (unfortunate, curve, government)  17.469419
29   (tough, difficult, humiliating)  17.469419


d:\Internship_Task\NewsTranscriptsTask\transcriptanalysis\preprocess.py:355: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tagged_list = ngram_df[to_keep][ngram_df['pmi'] > min_pmi]


                ngram      pmi
0   (affairs, author)  8.73471
48   (rapids, moment)  8.73471
56  (sort, struggles)  8.73471
55   (somehow, doesn)  8.73471
54     (solar, panel)  8.73471


d:\Internship_Task\NewsTranscriptsTask\transcriptanalysis\preprocess.py:355: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tagged_list = ngram_df[to_keep][ngram_df['pmi'] > min_pmi]


In [17]:
trigrams

16                    (idea, first, place)
1                 (author, caught, middle)
30        (unfortunate, curve, government)
29         (tough, difficult, humiliating)
28                   (terms, ready, world)
27               (tell, stories, continue)
26                 (solar, panel, factory)
25                 (same, sort, struggles)
24           (rapids, moment, interesting)
23               (place, rest, assumption)
21             (middle, phenomena, member)
20                 (member, station, wbez)
19    (manufacturing, american, byproduct)
18        (issue, manufacturing, american)
17                   (insane, idea, first)
22            (phenomena, member, station)
15                 (grand, rapids, moment)
8               (country, years, frontier)
14               (global, affairs, author)
3           (bulldozers, afternoon, knock)
4              (caught, middle, phenomena)
5           (change, immediate, aftermath)
6                     (cold, terms, ready)
7          

In [18]:
ngrams = list(trigrams) + list(bigrams)

In [19]:
len(qa_df['questions'])

50

# Feature Vectorization

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

n_wd_ngrams = np.empty((len(ngrams), len(qa_df['questions'])))

for i in range(len(ngrams)):
    for j in range(len(qa_df['questions'])):
        # print(ngrams[i])
        n_wd_ngrams[i][j] = qa_df.loc[j, 'questions'].count(' '.join(ngrams[i]))
        
cv = CountVectorizer(max_features = 32, stop_words='english')
n_wd = np.array(cv.fit_transform(qa_df['questions']).todense()).T
vocabulary = cv.get_feature_names()

n_wd = np.concatenate((n_wd, n_wd_ngrams))
ngrams = [' '.join(x) for x in ngrams]
vocabulary += ngrams

D:\Conda\envs\newenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
vocabulary[:10]

['allen',
 'barry',
 'best',
 'bring',
 'city',
 'community',
 'company',
 'did',
 'does',
 'feel']

# Hierarchical Topic Modelling 

In [22]:
! pip install protobuf==3.0.0
! pip install tqdm

In [23]:
# ! set PATH=%PATH%;D:\Conda\envs\newenv\Lib\BigARTM\bigartm-0.10.1\bin
# ! set PYTHONPATH=%PYTHONPATH%;D:\Conda\envs\newenv\Lib\BigARTM\bigartm-0.10.1\python
# ! cd D:\Conda\envs\newenv\Lib\BigARTM\bigartm-0.10.1\python

ModuleNotFoundError: No module named 'artm'

# Agglomerative Clustering